Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## First do regularized logistic regression
Note default is to apply regularization via the C (default=1.0). C is inversely proportional to the strength of the regularization. See Assignment 1

# Now try regularized neural net (with a hidden layer). I will use the batch version.

In [5]:
batch_size = 100
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
    
  # first the hidden layer
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

  # now the output layer
  biases_out = tf.Variable(tf.zeros([num_labels]))
  weights_out = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))  
    
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(hidden, weights_out) + biases_out
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  l2_reg = tf.nn.l2_loss(weights_out)
  loss += l2_reg * 1e-2 # adding l2 regularization
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
    
  valid_prediction = tf.nn.softmax(
    tf.matmul(
    tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden),
    weights_out) + biases_out)

  test_prediction = tf.nn.softmax(
    tf.matmul(
    tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden),
    weights_out) + biases_out)

In [6]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 369.933655
Minibatch accuracy: 10.0%
Validation accuracy: 29.9%
Minibatch loss at step 500: 14.847972
Minibatch accuracy: 74.0%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 5.439051
Minibatch accuracy: 79.0%
Validation accuracy: 77.8%
Minibatch loss at step 1500: 7.076602
Minibatch accuracy: 83.0%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 6.777390
Minibatch accuracy: 77.0%
Validation accuracy: 74.9%
Minibatch loss at step 2500: 6.036966
Minibatch accuracy: 78.0%
Validation accuracy: 77.5%
Minibatch loss at step 3000: 5.773693
Minibatch accuracy: 81.0%
Validation accuracy: 78.4%
Minibatch loss at step 3500: 3.709137
Minibatch accuracy: 75.0%
Validation accuracy: 79.4%
Minibatch loss at step 4000: 4.655849
Minibatch accuracy: 77.0%
Validation accuracy: 79.2%
Minibatch loss at step 4500: 2.494158
Minibatch accuracy: 83.0%
Validation accuracy: 81.3%
Test accuracy: 85.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 3001
batch_size = 100
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step % 10 * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 437.710968
Minibatch accuracy: 13.0%
Validation accuracy: 33.1%
Minibatch loss at step 500: 141.134537
Minibatch accuracy: 47.0%
Validation accuracy: 54.5%
Minibatch loss at step 1000: 0.169617
Minibatch accuracy: 99.0%
Validation accuracy: 78.1%
Minibatch loss at step 1500: 0.097245
Minibatch accuracy: 99.0%
Validation accuracy: 78.1%
Minibatch loss at step 2000: 0.262138
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 0.410438
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 3000: 0.025007
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Test accuracy: 85.5%


Note we are training on the same (almost) dataset on each batch due to the modulo 10 in the offset assignment.
We get 100% on the minibatch, less so on the test/validation.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

I will use the same as Q1, though without (and testing with) the l2 regularization on the loss.
The baseline vs l2 regularization is 88% on the test set.

In [8]:
batch_size = 250
hidden_nodes = 2000 #1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  keep_prob = tf.placeholder(tf.float32) # keep prob for dropout
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
    
  # first the hidden layer
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
  hidden_dropout = tf.nn.dropout(hidden, keep_prob=keep_prob)

  # now the output layer
  biases_out = tf.Variable(tf.zeros([num_labels]))
  weights_out = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))  
    
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(hidden_dropout, weights_out) + biases_out
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  #l2_reg = tf.nn.l2_loss(weights_out)
  #loss += l2_reg * 1e-2 # adding l2 regularization
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
    
  valid_prediction = tf.nn.softmax(
    tf.matmul(
    tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden),
    weights_out) + biases_out)

  test_prediction = tf.nn.softmax(
    tf.matmul(
    tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden),
    weights_out) + biases_out)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    
    feed_dict = {tf_train_dataset : batch_data,
                 tf_train_labels : batch_labels,
                 keep_prob : 0.9}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 521.914795
Minibatch accuracy: 8.4%
Validation accuracy: 27.5%
Minibatch loss at step 500: 8.905381
Minibatch accuracy: 85.2%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 18.074171
Minibatch accuracy: 79.2%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 10.975286
Minibatch accuracy: 85.2%
Validation accuracy: 84.3%
Minibatch loss at step 2000: 8.468433
Minibatch accuracy: 87.2%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 8.493932
Minibatch accuracy: 84.0%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 4.293402
Minibatch accuracy: 87.6%
Validation accuracy: 85.0%
Test accuracy: 91.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [10]:
batch_size = 250
hidden_nodes = 3000 #1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  keep_prob = tf.placeholder(tf.float32) # keep prob for dropout
  global_step = tf.Variable(0, trainable=False) # step for decaying learning rate
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps=10000, decay_rate=0.65, staircase=True)
  
  # first the hidden layer
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]))
    
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
  hidden_dropout = tf.nn.dropout(hidden, keep_prob=keep_prob)

  # now the output layer
  biases_out = tf.Variable(tf.zeros([num_labels]))
  weights_out = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))  
    
  logits = tf.matmul(hidden_dropout, weights_out) + biases_out
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  train_prediction = tf.nn.softmax(logits)
    
  valid_prediction = tf.nn.softmax(
    tf.matmul(
    tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden),
    weights_out) + biases_out)

  test_prediction = tf.nn.softmax(
    tf.matmul(
    tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden),
    weights_out) + biases_out)

In [11]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    
    feed_dict = {tf_train_dataset : batch_data,
                 tf_train_labels : batch_labels,
                 keep_prob : 0.85}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 608.665955
Minibatch accuracy: 10.4%
Validation accuracy: 30.3%
Minibatch loss at step 500: 13.633788
Minibatch accuracy: 87.2%
Validation accuracy: 84.7%
Minibatch loss at step 1000: 32.844830
Minibatch accuracy: 82.0%
Validation accuracy: 84.3%
Minibatch loss at step 1500: 20.190376
Minibatch accuracy: 84.4%
Validation accuracy: 85.4%
Minibatch loss at step 2000: 13.451656
Minibatch accuracy: 86.0%
Validation accuracy: 85.9%
Test accuracy: 92.1%


# Now try a deeper net with relu, dropout!

In [12]:
batch_size = 750
hidden_nodes1 = 3000 #1024
hidden_nodes2 = 500

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  keep_prob = tf.placeholder(tf.float32) # keep prob for dropout
  global_step = tf.Variable(0, trainable=False) # step for decaying learning rate
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps=1000, decay_rate=0.65, staircase=True)
  
  # the first hidden layer
  weights_hidden1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes1], stddev= 2.0/image_size/image_size))
  biases_hidden1 = tf.Variable(tf.zeros([hidden_nodes1]))
    
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1) # try sigmoid here?
  hidden_dropout1 = tf.nn.dropout(hidden1, keep_prob=keep_prob)
    
  # the second hidden layer
  weights_hidden2 = tf.Variable(
    tf.truncated_normal([hidden_nodes1, hidden_nodes2], stddev=np.sqrt(2.0/hidden_nodes1)))
  biases_hidden2 = tf.Variable(tf.zeros([hidden_nodes2]))
    
  hidden2 = tf.nn.relu(tf.matmul(hidden_dropout1, weights_hidden2) + biases_hidden2)
  hidden_dropout2 = tf.nn.dropout(hidden2, keep_prob=keep_prob)

  # now the output layer
  biases_out = tf.Variable(tf.zeros([num_labels]))
  weights_out = tf.Variable(tf.truncated_normal([hidden_nodes2, num_labels], stddev=np.sqrt(2.0/hidden_nodes2)))  
    
  logits = tf.matmul(hidden_dropout2, weights_out) + biases_out
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  train_prediction = tf.nn.softmax(logits)
    
  layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1)
  layer2_valid = tf.nn.relu(tf.matmul(layer1_valid, weights_hidden2) + biases_hidden2)
  valid_prediction = tf.nn.softmax(tf.matmul(layer2_valid, weights_out) + biases_out)

  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1)
  layer2_test = tf.nn.relu(tf.matmul(layer1_test, weights_hidden2) + biases_hidden2)
  test_prediction = tf.nn.softmax(tf.matmul(layer2_test, weights_out) + biases_out)

In [13]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    
    feed_dict = {tf_train_dataset : batch_data,
                 tf_train_labels : batch_labels,
                 keep_prob : 0.95}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.302102
Minibatch accuracy: 9.1%
Validation accuracy: 59.9%
Minibatch loss at step 500: 0.384513
Minibatch accuracy: 88.5%
Validation accuracy: 87.8%
Minibatch loss at step 1000: 0.312597
Minibatch accuracy: 90.8%
Validation accuracy: 89.4%
Minibatch loss at step 1500: 0.262691
Minibatch accuracy: 91.7%
Validation accuracy: 90.3%
Minibatch loss at step 2000: 0.237210
Minibatch accuracy: 93.9%
Validation accuracy: 91.1%
Minibatch loss at step 2500: 0.196151
Minibatch accuracy: 94.8%
Validation accuracy: 91.1%
Minibatch loss at step 3000: 0.171267
Minibatch accuracy: 94.7%
Validation accuracy: 91.2%
Minibatch loss at step 3500: 0.194088
Minibatch accuracy: 93.9%
Validation accuracy: 91.2%
Minibatch loss at step 4000: 0.153029
Minibatch accuracy: 95.9%
Validation accuracy: 91.4%
Minibatch loss at step 4500: 0.136731
Minibatch accuracy: 96.3%
Validation accuracy: 91.7%
Minibatch loss at step 5000: 0.150050
Minibatch accuracy: 95.5%
Validation accuracy

In [16]:
# Now see if can do the above with keras!

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam, SGD

In [42]:
model = Sequential()
model.add(Dense(hidden_nodes1, input_dim=image_size **2, activation='relu', init='he_normal', bias=True))
model.add(Dropout(0.05))
model.add(Dense(hidden_nodes2, activation='relu', init='he_normal', bias=True))
model.add(Dropout(0.05))
model.add(Dense(10, init='he_normal', activation='softmax', bias=True))

sgd = SGD(lr=0.1, nesterov=True, decay=1e-8)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [43]:
model.fit(train_dataset, train_labels, nb_epoch=10, batch_size=250)

Epoch 1/10
200000/200000 [==============================] - 121s - loss: 0.5302 - acc: 0.8441   
Epoch 2/10
200000/200000 [==============================] - 116s - loss: 0.4038 - acc: 0.8800   
Epoch 3/10
200000/200000 [==============================] - 118s - loss: 0.3545 - acc: 0.8943   
Epoch 4/10
200000/200000 [==============================] - 119s - loss: 0.3198 - acc: 0.9047   
Epoch 5/10
200000/200000 [==============================] - 119s - loss: 0.2919 - acc: 0.9123   
Epoch 6/10
200000/200000 [==============================] - 119s - loss: 0.2692 - acc: 0.9194   
Epoch 7/10
200000/200000 [==============================] - 119s - loss: 0.2482 - acc: 0.9255   
Epoch 8/10
200000/200000 [==============================] - 119s - loss: 0.2287 - acc: 0.9316   
Epoch 9/10
200000/200000 [==============================] - 119s - loss: 0.2105 - acc: 0.9367   
Epoch 10/10
200000/200000 [==============================] - 120s - loss: 0.1946 - acc: 0.9420   


In [41]:
train_score = model.evaluate(train_dataset, train_labels, batch_size=20000)
print(train_score)
test_score = model.evaluate(test_dataset, test_labels, batch_size=1000)
print(test_score)

200000/200000 [==============================] - 44s    
[0.075133423507213595, 0.97886999845504763]
10000/10000 [==============================] - 2s     
[0.14990556463599206, 0.9592999935150146]


# About the same results